# ওপেন এআই মডেল ফাইন টিউনিং

এই নোটবুকটি ওপেন এআই-এর [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) ডকুমেন্টেশনের বর্তমান নির্দেশনার উপর ভিত্তি করে তৈরি।

ফাইন-টিউনিং আপনার অ্যাপ্লিকেশনের জন্য ফাউন্ডেশন মডেলের পারফরম্যান্স উন্নত করে, কারণ এতে নির্দিষ্ট ব্যবহার ক্ষেত্র বা পরিস্থিতির সাথে সম্পর্কিত অতিরিক্ত ডেটা ও কনটেক্সট দিয়ে মডেলটিকে পুনরায় প্রশিক্ষণ দেওয়া হয়। মনে রাখবেন, _few shot learning_ এবং _retrieval augmented generation_-এর মতো প্রম্পট ইঞ্জিনিয়ারিং কৌশলগুলো আপনাকে ডিফল্ট প্রম্পটের সাথে প্রাসঙ্গিক ডেটা যোগ করে কোয়ালিটি বাড়াতে সাহায্য করে। তবে, এই পদ্ধতিগুলো টার্গেট ফাউন্ডেশন মডেলের সর্বোচ্চ টোকেন উইন্ডো সাইজ দ্বারা সীমাবদ্ধ।

ফাইন-টিউনিংয়ের মাধ্যমে, আমরা কার্যত মডেলটিকেই প্রয়োজনীয় ডেটা দিয়ে পুনরায় প্রশিক্ষণ দিচ্ছি (এতে আমরা অনেক বেশি উদাহরণ ব্যবহার করতে পারি, যা সর্বোচ্চ টোকেন উইন্ডোতে সম্ভব নয়) এবং একটি _কাস্টম_ সংস্করণের মডেল ডিপ্লয় করছি, যেটিকে আর ইনফারেন্সের সময় উদাহরণ দিতে হয় না। এতে শুধু আমাদের প্রম্পট ডিজাইনের কার্যকারিতা বাড়ে (আমরা টোকেন উইন্ডো আরও নমনীয়ভাবে ব্যবহার করতে পারি), বরং খরচও কমতে পারে (কারণ ইনফারেন্সের সময় মডেলে পাঠানো টোকেনের সংখ্যা কমে যায়)।

ফাইন-টিউনিংয়ের ৪টি ধাপ:
1. ট্রেনিং ডেটা প্রস্তুত ও আপলোড করুন।
1. ট্রেনিং জব চালিয়ে ফাইন-টিউনড মডেল পান।
1. ফাইন-টিউনড মডেল মূল্যায়ন করুন এবং কোয়ালিটির জন্য পুনরাবৃত্তি করুন।
1. সন্তুষ্ট হলে ইনফারেন্সের জন্য ফাইন-টিউনড মডেল ডিপ্লয় করুন।

মনে রাখবেন, সব ফাউন্ডেশন মডেল ফাইন-টিউনিং সাপোর্ট করে না - সর্বশেষ তথ্যের জন্য [OpenAI ডকুমেন্টেশন দেখুন](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)। আপনি চাইলে পূর্বে ফাইন-টিউন করা মডেলও আবার ফাইন-টিউন করতে পারেন। এই টিউটোরিয়ালে আমরা `gpt-35-turbo` কে আমাদের টার্গেট ফাউন্ডেশন মডেল হিসেবে ফাইন-টিউন করব।

---


### ধাপ ১.১: আপনার ডেটাসেট প্রস্তুত করুন

চলুন একটি চ্যাটবট তৈরি করি, যা আপনাকে মৌলিক উপাদানগুলোর পর্যায় সারণি বুঝতে সাহায্য করবে এবং প্রতিটি উপাদান সম্পর্কে প্রশ্নের উত্তর দিবে একটি লিমেরিক আকারে। _এই_ সহজ টিউটোরিয়ালে, আমরা শুধু একটি ডেটাসেট তৈরি করব, যাতে কিছু নমুনা উত্তর থাকবে, যা ডেটার প্রত্যাশিত ফরম্যাট দেখাবে। বাস্তব জীবনের ক্ষেত্রে, আপনাকে আরও অনেক বেশি নমুনা দিয়ে একটি ডেটাসেট তৈরি করতে হবে। আপনি চাইলে কোনো ওপেন ডেটাসেট (আপনার অ্যাপ্লিকেশন ডোমেইনের জন্য) থাকলে সেটিও ব্যবহার করতে পারেন এবং ফাইন-টিউনিংয়ের জন্য সেটি রিফরম্যাট করতে পারেন।

আমরা যেহেতু `gpt-35-turbo`-এর উপর ফোকাস করছি এবং একক-টার্ন (single-turn) উত্তর চাই (chat completion), তাই আমরা [এই পরামর্শকৃত ফরম্যাট](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) ব্যবহার করে উদাহরণ তৈরি করতে পারি, যা OpenAI চ্যাট কমপ্লিশন-এর চাহিদা পূরণ করে। আপনি যদি বহু-টার্ন কথোপকথনের কনটেন্ট আশা করেন, তাহলে আপনাকে [বহু-টার্ন উদাহরণ ফরম্যাট](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) ব্যবহার করতে হবে, যেখানে একটি `weight` প্যারামিটার থাকে, যা নির্দেশ করে কোন মেসেজগুলো ফাইন-টিউনিং প্রক্রিয়ায় ব্যবহার হবে (বা হবে না)।

আমরা এখানে আমাদের টিউটোরিয়ালের জন্য সহজ একক-টার্ন ফরম্যাট ব্যবহার করব। ডেটা থাকবে [jsonl ফরম্যাটে](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), যেখানে প্রতি লাইনে ১টি রেকর্ড থাকবে, প্রতিটিই JSON-ফরম্যাটেড অবজেক্ট হিসেবে। নিচের স্নিপেটে ২টি রেকর্ড দেখানো হয়েছে নমুনা হিসেবে - সম্পূর্ণ নমুনা সেট (১০টি উদাহরণ) দেখতে [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) দেখুন, যা আমরা আমাদের ফাইন-টিউনিং টিউটোরিয়ালে ব্যবহার করব। **নোট:** প্রতিটি রেকর্ড _অবশ্যই_ এক লাইনে সংজ্ঞায়িত হতে হবে (ফরম্যাটেড JSON ফাইলের মতো একাধিক লাইনে বিভক্ত নয়)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

বাস্তব জীবনের ক্ষেত্রে ভালো ফলাফলের জন্য আপনাকে অনেক বড় উদাহরণ সেট লাগবে - এখানে গুণগত মান ও ফাইন-টিউনিংয়ের সময়/খরচের মধ্যে ভারসাম্য রাখতে হবে। আমরা ছোট সেট ব্যবহার করছি, যাতে দ্রুত ফাইন-টিউনিং শেষ করে প্রক্রিয়াটি দেখাতে পারি। আরও জটিল ফাইন-টিউনিং টিউটোরিয়ালের জন্য [এই OpenAI Cookbook উদাহরণটি](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) দেখুন।


### ধাপ ১.২ আপনার ডেটাসেট আপলোড করুন

Files API ব্যবহার করে ডেটা আপলোড করুন [এখানে বর্ণিতভাবে](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)। মনে রাখবেন, এই কোড চালানোর আগে আপনাকে নিচের ধাপগুলো সম্পন্ন করতে হবে:
 - `openai` পাইথন প্যাকেজ ইনস্টল করতে হবে (নিশ্চিত করুন আপনি >=0.28.0 ভার্সন ব্যবহার করছেন, যাতে সর্বশেষ ফিচারগুলো পান)
 - `OPENAI_API_KEY` এনভায়রনমেন্ট ভ্যারিয়েবল হিসেবে আপনার OpenAI API কী সেট করতে হবে
আরও জানতে, কোর্সের জন্য দেওয়া [সেটআপ গাইড](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) দেখুন।

এখন, আপনার লোকাল JSONL ফাইল থেকে আপলোডের জন্য একটি ফাইল তৈরি করতে নিচের কোডটি চালান।


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### ধাপ ২.১: SDK ব্যবহার করে ফাইন-টিউনিং জব তৈরি করুন


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### ধাপ ২.২: জবের স্ট্যাটাস চেক করুন

`client.fine_tuning.jobs` API দিয়ে আপনি কিছু কাজ করতে পারেন:
- `client.fine_tuning.jobs.list(limit=<n>)` - সর্বশেষ nটি ফাইন-টিউনিং জবের তালিকা দেখুন
- `client.fine_tuning.jobs.retrieve(<job_id>)` - নির্দিষ্ট কোনো ফাইন-টিউনিং জবের বিস্তারিত দেখুন
- `client.fine_tuning.jobs.cancel(<job_id>)` - কোনো ফাইন-টিউনিং জব বাতিল করুন
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - জব থেকে সর্বোচ্চ nটি ইভেন্টের তালিকা দেখুন
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

প্রক্রিয়ার প্রথম ধাপ হলো _ট্রেনিং ফাইল যাচাই করা_, যাতে ডেটা সঠিক ফরম্যাটে আছে কি না তা নিশ্চিত করা যায়।


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### ধাপ ২.৩: অগ্রগতি পর্যবেক্ষণের জন্য ইভেন্ট ট্র্যাক করুন


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### ধাপ ২.৪: ওপেনএআই ড্যাশবোর্ডে স্ট্যাটাস দেখুন


আপনি OpenAI ওয়েবসাইটে গিয়ে _Fine-tuning_ সেকশনটি এক্সপ্লোর করেও স্ট্যাটাস দেখতে পারেন। এখানে আপনি বর্তমান জবের স্ট্যাটাস দেখতে পারবেন, এবং আগের জব এক্সিকিউশন রানগুলোর ইতিহাসও ট্র্যাক করতে পারবেন। এই স্ক্রিনশটে দেখা যাচ্ছে, আগের এক্সিকিউশনটি ব্যর্থ হয়েছে, এবং দ্বিতীয় রানটি সফল হয়েছে। প্রসঙ্গত, প্রথম রানটি একটি ভুল ফরম্যাট করা রেকর্ডসহ JSON ফাইল ব্যবহার করেছিল - সেটি ঠিক করার পর দ্বিতীয় রানটি সফলভাবে সম্পন্ন হয়েছে এবং মডেলটি ব্যবহারের জন্য উপলব্ধ হয়েছে।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.bn.png)


আপনি ভিজ্যুয়াল ড্যাশবোর্ডে নিচে স্ক্রল করেও স্ট্যাটাস মেসেজ এবং মেট্রিক্স দেখতে পারেন:

| মেসেজ | মেট্রিক্স |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.bn.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.bn.png)|


### ধাপ ৩.১: আইডি সংগ্রহ করুন এবং কোডে ফাইন-টিউন করা মডেলটি পরীক্ষা করুন


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### ধাপ ৩.২: প্লেগ্রাউন্ডে ফাইন-টিউনড মডেল লোড ও পরীক্ষা করুন

এখন আপনি দুইভাবে ফাইন-টিউনড মডেলটি পরীক্ষা করতে পারেন। প্রথমত, আপনি প্লেগ্রাউন্ডে যেতে পারেন এবং মডেলস ড্রপ-ডাউন থেকে আপনার নতুন ফাইন-টিউনড মডেলটি তালিকা থেকে নির্বাচন করতে পারেন। আরেকটি উপায় হলো ফাইন-টিউনিং প্যানেলে দেখানো "Playground" অপশনটি ব্যবহার করা (উপরের স্ক্রিনশট দেখুন), যা একটি _তুলনামূলক_ ভিউ চালু করে যেখানে ফাউন্ডেশন এবং ফাইন-টিউনড মডেলের সংস্করণগুলো পাশাপাশি দেখানো হয় দ্রুত মূল্যায়নের জন্য।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.bn.png)

আপনার ট্রেনিং ডেটায় ব্যবহৃত সিস্টেম কনটেক্সটটি এখানে দিন এবং আপনার টেস্ট প্রশ্নটি লিখুন। আপনি দেখবেন, উভয় পাশে একই কনটেক্সট ও প্রশ্ন আপডেট হয়েছে। তুলনা চালান এবং দেখুন, তাদের আউটপুটে কী পার্থক্য হচ্ছে। _লক্ষ্য করুন, ফাইন-টিউনড মডেলটি আপনার উদাহরণে দেয়া ফরম্যাটে উত্তর দিচ্ছে, যেখানে ফাউন্ডেশন মডেল কেবল সিস্টেম প্রম্পট অনুসরণ করছে_।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.bn.png)

আপনি লক্ষ্য করবেন, তুলনায় প্রতিটি মডেলের জন্য টোকেন সংখ্যা এবং ইনফারেন্সে সময় কত লাগছে, সেটিও দেখানো হচ্ছে। **এই নির্দিষ্ট উদাহরণটি খুবই সাধারণ, কেবল প্রক্রিয়াটি দেখানোর জন্য, বাস্তব কোনো ডেটাসেট বা পরিস্থিতি তুলে ধরার জন্য নয়**। আপনি দেখতে পাবেন, দুই নমুনাতেই টোকেন সংখ্যা একই (কারণ সিস্টেম কনটেক্সট ও ইউজার প্রম্পট এক), তবে ফাইন-টিউনড মডেলে ইনফারেন্সে একটু বেশি সময় লাগছে (কারণ এটি কাস্টম মডেল)।

বাস্তব পরিস্থিতিতে, আপনি এ ধরনের খেলনা উদাহরণ ব্যবহার করবেন না, বরং বাস্তব ডেটার (যেমন, কাস্টমার সার্ভিসের জন্য প্রোডাক্ট ক্যাটালগ) ওপর ফাইন-টিউন করবেন, যেখানে উত্তরগুলোর মান অনেক বেশি স্পষ্ট হবে। _সেই_ প্রেক্ষাপটে, ফাউন্ডেশন মডেল দিয়ে সমমানের উত্তর পেতে হলে আরও বেশি কাস্টম প্রম্পট ইঞ্জিনিয়ারিং করতে হবে, এতে টোকেন ব্যবহারের পরিমাণ ও ইনফারেন্সের সময় বাড়তে পারে। _এটা চেষ্টা করে দেখতে চাইলে, OpenAI Cookbook-এ থাকা ফাইন-টিউনিং উদাহরণগুলো দেখে শুরু করুন_।



---

**দায়িত্ব অস্বীকার**:
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব নির্ভুলতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে। মূল ভাষায় রচিত নথিটিকেই চূড়ান্ত ও নির্ভরযোগ্য উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদের ব্যবহারে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়ী থাকব না।
